In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [9]:
#folder = "../../Data/hpc_out/rwm_wip_2_6r7m/"
mol_type = "6r7m"
folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"

id_index = 1
separator = "."
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println("Error in $file")
        end
    end
end

In [10]:
argmin(min_Es)

52

In [11]:
for elem in evaluation_strings
    println(elem)
end

1: 2.0 | 0.561144 | 116.2579343156045, -74.39070874849418, -70.02489655084301 | 5.465894079549043
2: 2.0 | 0.5199005 | -27.779613410898534, -76.3142186946791, -72.22509511841722 | 17.899082660306043
3: 2.0 | 0.5245148 | 155.73017615795382, -71.28645513603374, -67.88636903244657 | 7.301147158098653
4: 2.0 | 0.50511247 | 58.21210228422587, -75.41793849036424, -64.51699766551934 | 19.242491063574743
5: 2.0 | 0.5164776 | 122.3832698945532, -74.8612083409034, -67.52276927189405 | 3.7982639529333104
6: 2.0 | 0.5799458 | 102.46704073002707, -75.30653338745873, -71.27858180741333 | 9.018541440834944
7: 2.0 | 0.5463228 | -11.32424242263476, -75.22971562615625, -57.731838002046686 | 17.048642364612828
8: 2.0 | 0.561127 | 125.64576292990907, -76.8341669939573, -64.7544802099388 | 9.639734759446755
9: 2.0 | 0.61335844 | 144.8908382035396, -71.87417457768296, -65.57745940088728 | 7.114299779962915
10: 2.0 | 0.4990663 | 232.22181981951059, -74.73491496932816, -73.82655411319922 | 4.206755993723197
1

In [6]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [34]:
@load "../../Data/hpc_out/rwm_op_3_6r7m/5_rwm_op_3_6r7m.jld2" input output


2-element Vector{Symbol}:
 :input
 :output

In [35]:
input["template_centers"]

3×1206 Matrix{Float64}:
 -27.8796   -26.6674   -26.5597   …  -31.3903   -31.5191   -32.6934
   9.063      9.54921    9.03875       8.50467    8.25724    9.091
  -4.79258   -4.1493    -2.71995      -1.62113   -0.2168    -2.14403

In [36]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
